# Configurando Apache en CentOS para ejecutar aplicaciones Python con WSGI

Se asumen los siguientes requisitos:

1. Distribucón de Linux CentOS 7
1. Servidor web Apache tiene instalado y respondiendo peticiones en el puerto 80
1. Firewall con acceso puerto 80 (o cualquier otro puerto usado) desde cualquier equipo en la LAN
1. Usuario creado específicamente para colocar las aplicaciones web en Python (rctorr en este caso)
1. El paquete `httpd-devel` debe estar instalado en CentOS

### Subiendo la APP

Dentro de la carpeta home del usuario se crea la carpeta `webapps`

```bash
$ mkdir webapps
$ cd webapps
$ _
````

Luego subir la carpeta `app` de la sesión anterior y colocarla dentro de la carpeta `webapps`
```bash
[rctorr@centos webapps]$ tree
.
`-- app
    |-- holamundo.py
    `-- info.py

1 directory, 2 files
[rctorr@centos webapps]$ 
```

### Instalando Miniconda3

Se descarga Miniconda y se instala con el usuario que se dedicará a ejecutar las aplicaciones web en Python.

```bash
[rctorr@centos ~]$ sh Miniconda3-latest-Linux-x86_64.sh
...
installation finished.
Do you wish the installer to prepend the Miniconda3 install location
to PATH in your /home/rctorr/.bashrc ? [yes|no]
[no] >>> yes

[rctorr@centos ~]$ _
```

Posteriormente, es necesario cerrar la terminar y entrar nuevamente y entonces ejecutar:

```bash
[rctorr@centos ~]$ conda -V
conda 4.5.11
[rctorr@centos ~]$ 
```

Con esto tenemos instalado miniconda

### Instalando el módulo `mod_wsgi` en CentOS en Miniconda3

Se instala el módulo `mod_wsgi` usando `pip`

```bash
[rctorr@centos ~]$ pip install mod_wsgi
Collecting mod_wsgi
  Using cached https://files.pythonhosted.org/packages/9e/37/dd336068ece37c43957aa337f25c59a9a6afa98086e5507908a2d21ab807/mod_wsgi-4.6.4.tar.gz
Building wheels for collected packages: mod-wsgi
  Running setup.py bdist_wheel for mod-wsgi ... done
  Stored in directory: /home/rctorr/.cache/pip/wheels/2d/73/68/9dcbbd0147b3fde4686263a31324ea2372e42f7cefa2f7d181
Successfully built mod-wsgi
Installing collected packages: mod-wsgi
Successfully installed mod-wsgi-4.6.4
[rctorr@centos ~]$ 
```

Para verificar que el módulo está funcionando adecuadamente se ejecutará un procesos de Apache independiente del que ya se está corriendo para hacer funcionar la palicación `hola_mundo_app.py`.

``` bash
$ cd webapps/app
webapps/app $ mod_wsgi-express start-server --host 0.0.0.0 holamundo.py \
--callable-object hola_mundo_app
Server URL         : http://0.0.0.0:8000/
Server Root        : /tmp/mod_wsgi-0.0.0.0:8000:1000
Server Conf        : /tmp/mod_wsgi-0.0.0.0:8000:1000/httpd.conf
Error Log File     : /tmp/mod_wsgi-0.0.0.0:8000:1000/error_log (warn)
Request Capacity   : 5 (1 process * 5 threads)
Request Timeout    : 60 (seconds)
Startup Timeout    : 15 (seconds)
Queue Backlog      : 100 (connections)
Queue Timeout      : 45 (seconds)
Server Capacity    : 20 (event/worker), 20 (prefork)
Server Backlog     : 500 (connections)
Locale Setting     : es_US.UTF-8
```

Lo anterior inicia un proceso de Apache en el puerto 8000 por default y crea toda una carpta de configuración para Apache en la carpeta `/tmp/mod_wsgi-0.0.0.0:8000:1000`, todo esto es temporal y es para verificar el funcionamiento del módulo.

Abrir en el navegador alguna de las urls:
- http://localhost:8000/
- http://IPADDRESS:8000/

Para cancelar se presiona Control+C

Ahora ejecutar la aplicación `info.py` con el comando:

``` bash
webapps/app $ mod_wsgi-express start-server --host 0.0.0.0 info.py --callable-object \
info_app --reload-on-changes
Server URL         : http://0.0.0.0:8000/
Server Root        : /tmp/mod_wsgi-0.0.0.0:8000:1000
Server Conf        : /tmp/mod_wsgi-0.0.0.0:8000:1000/httpd.conf
Error Log File     : /tmp/mod_wsgi-0.0.0.0:8000:1000/error_log (warn)
Request Capacity   : 5 (1 process * 5 threads)
Request Timeout    : 60 (seconds)
Startup Timeout    : 15 (seconds)
Queue Backlog      : 100 (connections)
Queue Timeout      : 45 (seconds)
Server Capacity    : 20 (event/worker), 20 (prefork)
Server Backlog     : 500 (connections)
Locale Setting     : es_US.UTF-8
```

Abrir en el navegador alguna de las urls:
- http://localhost:8000/
- http://IPADDRESS:8000/

Notar que en la opción `--callable-object info_app` se está indicando que para iniciar la aplicación hay que llamar al método `info_app()`, si se desea omitir esta opción, entonces el método se tiene que llamar `application()` y la opción `--reload-on-changes` lo que hace es ver si se ha modificado algunos de los archivos, si es así se recargar en automático para refejar los cambios, así que ya no es necesario estár presionando Control+C y ejecutando de nuevo.

![imagen: Resultado de la ejecución](mod_wsgi-info.py.png)

### Subiendo la bottle_app

Subir la carpeta `bottle_app` y colocarla dentro de la carpeta `webapps`
```bash
[rctorr@centos webapps]$ tree
.
├── app
│   ├── holamundo.py
│   └── info.py
└── bottle_app
    ├── bottle.py
    ├── formulario-bottle.py
    ├── holamundo.py
    ├── img
    │   └── python-logo.png
    └── template
        ├── formulario.html
        └── formulario_resp.html

5 directories, 13 files
[rctorr@centos webapps]$ 
```

Para ejecutar la aplicación `bottle_app/holamundo.py` hay que crear el archivo `holamundo.wsgi` que también es un script en Python y tendrá el siguiente contenido:

``` python
import bottle
import os

os.chdir(os.path.dirname(__file__))

"""
Aquí creamos o importamos nuestra aplicación y no se tiene que usar run()
porque el servidor ahora es externo a nuestro script.
"""
import holamundo

# Definimos application como lo pide el estándar WSGI
application = bottle.default_app()
```

Lo guardamos y lo subimos al servidor también, entonces nos cambiamos a la carpeta `webapps/bottle_app` y ejecutamos lo siguiente:

``` bash
[rctorr@centos bottle_app]$ mod_wsgi-express start-server --host 0.0.0.0 holamundo.wsgi --reload-on-changes
Server URL         : http://0.0.0.0:8000/
Server Root        : /tmp/mod_wsgi-0.0.0.0:8000:1000
Server Conf        : /tmp/mod_wsgi-0.0.0.0:8000:1000/httpd.conf
Error Log File     : /tmp/mod_wsgi-0.0.0.0:8000:1000/error_log (warn)
Request Capacity   : 5 (1 process * 5 threads)
Request Timeout    : 60 (seconds)
Startup Timeout    : 15 (seconds)
Queue Backlog      : 100 (connections)
Queue Timeout      : 45 (seconds)
Server Capacity    : 20 (event/worker), 20 (prefork)
Server Backlog     : 500 (connections)
Locale Setting     : es_US.UTF-8
```

Ahora es momento de abrir las siguientes urls:
- http://IP_ADDRESS:8000/
- http://IP_ADDRESS:8000/hola/rctorr

En la última url se debe observar lo siguinete:

![Resultado de la url /hola/rctorr](mod_wsgi-bottle_holamundo.py.png)

Para ejecutar la aplicación `bottle_app/formulario.py` hay que crear el archivo `formulario.wsgi` que también es un script en Python y tendrá el siguiente contenido:

``` python
import bottle
import os

os.chdir(os.path.dirname(__file__))

"""
Aquí creamos o importamos nuestra aplicación y no se tiene que usar run()
porque el servidor ahora es externo a nuestro script.
"""
import formulario

# Definimos application como lo pide el estándar WSGI
application = bottle.default_app()
```

Lo guardamos y lo subimos al servidor también, entonces nos cambiamos a la carpeta `webapps/bottle_app` y ejecutamos lo siguiente:

``` bash
[rctorr@centos bottle_app]$ mod_wsgi-express start-server --host 0.0.0.0 formulario.wsgi --reload-on-changes
Server URL         : http://0.0.0.0:8000/
Server Root        : /tmp/mod_wsgi-0.0.0.0:8000:1000
Server Conf        : /tmp/mod_wsgi-0.0.0.0:8000:1000/httpd.conf
Error Log File     : /tmp/mod_wsgi-0.0.0.0:8000:1000/error_log (warn)
Request Capacity   : 5 (1 process * 5 threads)
Request Timeout    : 60 (seconds)
Startup Timeout    : 15 (seconds)
Queue Backlog      : 100 (connections)
Queue Timeout      : 45 (seconds)
Server Capacity    : 20 (event/worker), 20 (prefork)
Server Backlog     : 500 (connections)
Locale Setting     : es_US.UTF-8
```

Ahora es momento de abrir las siguientes urls:
- http://IP_ADDRESS:8000/

El resultado debería de ser lo siguinete:

![Resultado de la url](mod_wsgi-bottle_formulario.py.png)

Hasta este punto se tiene ya instalado el módulo `mod_wsgi`, se tiene configurado el usuario y las carpetas para cada aplicación y también se han creado los script wsgi necesarios para iniciar las aplicaciones que los requieren y finalmente se ha verificado que las aplicaciones funcionan correctamente con Apache + WSGI.

Ahora sólo resta hacer la configuración para que las aplicaciones sean procesadas por el proceso principal de Apache en el puerto 80.

### Configurando Apache para que la aplicación inicial sea `bottle_app/holamundo.py`

La aplicación deberá ejecutarse cuando se usen algunas de las siguientes urls:
- http://IP_ADDRESS
- http://IP_ADDRESS/


### Agregando el módulo `mod_wsgi` instalado desde el ambiente de Miniconda3

Al revisar la instalación de CentOs 7, en la carpeta de módulos de Apache se observa lo siguiente:

```bash
[rctorr@centos ~]$ ls -lF /etc/httpd/conf.modules.d/
total 44
-rw-r--r--. 1 root root 3739 jun 26 13:07 00-base.conf
-rw-r--r--. 1 root root  139 jun 26 13:07 00-dav.conf
-rw-r--r--. 1 root root   41 jun 26 13:07 00-lua.conf
-rw-r--r--. 1 root root  742 jun 26 13:07 00-mpm.conf
-rw-r--r--. 1 root root  957 jun 26 13:07 00-proxy.conf
-rw-r--r--. 1 root root   41 jun 26 13:07 00-ssl.conf
-rw-r--r--. 1 root root   88 jun 26 13:07 00-systemd.conf
-rw-r--r--. 1 root root  451 jun 26 13:07 01-cgi.conf
-rw-r--r--. 1 root root   45 dic  1  2017 10-fcgid.conf
-rw-r--r--. 1 root root  226 sep 14 06:05 10-wsgi.conf
[rctorr@centos ~]$ 
```

lo que indica que ya existe un módulo wsgi configurado en el servidor, sin embargo ese módulo podría no estar actualizado o funcionar sólo con las versiones de Python instaladas en el sistema, por lo tanto vamos a abrir el archivo `10-wsgi.conf` y remplazar su contenido con el siguiente:

```
LoadModule wsgi_module "/home/rctorr/miniconda3/lib/python3.7/site-packages/mod_
wsgi/server/mod_wsgi-py37.cpython-37m-x86_64-linux-gnu.so"
WSGIPythonHome "/home/rctorr/miniconda3"
```

Notar que todas las rutas son absolutas y hacen referencia a la instalación de Miniconda3 y el módulo.

Ahora es necesario reiniciar Apache con:

``` bash
[rctorr@centos ~]$ sudo systemctl restart httpd
[sudo] password for rctorr: 
[rctorr@centos ~]$
```

### Haciendo que `bottle_app/holamundo.py` sea la aplicación de inicio

Agregar las siguientes líneas al final del archivo `httpd.conf`:

```
# Definimos algunos parámetro para wel módulo WSGI
WSGIDaemonProcess bottle_app python-path=/home/rctorr/webapps/bottle_app
WSGIProcessGroup bottle_app
# Al crear el alias a nuestra app usar rutas absolutas
WSGIScriptAlias / /home/rctorr/webapps/bottle_app/holamundo.wsgi
# Definiendo los permisos al directorio de nuestra app
<Directory /home/rctorr/webapps/bottle_app>
    # Sólo le damos permisos al archivo de nuestra app (paranóicos!)
    <Files holamundo.wsgi>
        Require all granted
    </Files>
</Directory>
```

Reiniciamos Apache:

``` bash
[rctorr@centos ~]$ sudo systemctl restart httpd
[rctorr@centos ~]$
```

Y accedemos a nuestra app:
- http://IP_ADDRESS/

donde IP_ADDRESS es la dirección IP del servidor

Se recomienda acceder a la siguientes urls y verificar funcionamiento:
- http://IP_ADDRESS/index.html
- http://IP_ADDRESS/hola/mi_nombre


### Agregando otra app en la ruta `/formulario`

Agregar las siguientes líneas al archivo `httpd.conf`:

```
# Se agrega la ruta a la segunda app
WSGIScriptAlias /formulario /home/rctorr/webapps/bottle_app/formulario.wsgi
# Al crear el alias a nuestra app usar rutas absolutas
WSGIScriptAlias / /home/rctorr/webapps/bottle_app/holamundo.wsgi
# Definiendo los permisos al directorio de nuestra app
<Directory /home/rctorr/webapps/bottle_app>
    # Sólo le damos permisos al archivo de nuestra app (paranóicos!)
    <Files holamundo.wsgi>
        Require all granted
    </Files>
    <Files formulario.wsgi>
        Require all granted
    </Files>
</Directory>
```

Notar que el alias para `/formulario` está antes que `/`, esto es importante

Reiniciamos Apache:

``` bash
[rctorr@centos ~]$ sudo systemctl restart httpd
[rctorr@centos ~]$
```

Y accedemos a nuestra app:
- http://IP_ADDRESS/formulario

Sin embargo aún tenemos un problema con la imagen!

### Agregando un alias para los archivos estáticos

Agregar las siguientes líneas al final del archivo `httpd.conf`:

```
# Se agrega el alias para los archivos estáticos
Alias /img /home/rctorr/webapps/bottle_app/img
<Directory /home/rctorr/webapps/bottle_app/img>
    Require all granted
</Directory>
```

Reiniciamos Apache:

``` bash
[rctorr@centos ~]$ sudo systemctl restart httpd
```

Y accedemos a nuestra app:
- http://IP_ADDRESS/formulario

Ahora se debería de ver la imágen en el encabezado.